## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-10-23-52-35 +0000,nypost,Eric Adams’ economic development honcho seeks ...,https://nypost.com/2025/11/10/us-news/eric-ada...
1,2025-11-10-23-49-34 +0000,nyt,Marriage Ruling Relieves Gay Americans and Lea...,https://www.nytimes.com/2025/11/10/us/supreme-...
2,2025-11-10-23-47-00 +0000,wsj,Gold Edges Higher on Signs of Possible End to ...,https://www.wsj.com/finance/commodities-future...
3,2025-11-10-23-42-42 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/11/10/us/tru...
4,2025-11-10-23-41-29 +0000,nyt,Age Is the Issue That Democrats Can’t Shut Down,https://www.nytimes.com/2025/11/10/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2377/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
240,trump,41
27,shutdown,32
92,government,19
18,new,17
93,deal,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
84,2025-11-10-21-11-40 +0000,nypost,Trump pledges to ‘abide’ by deal to end govern...,https://nypost.com/2025/11/10/us-news/trump-pl...,132
119,2025-11-10-19-39-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...,132
29,2025-11-10-22-53-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,116
230,2025-11-10-13-44-58 +0000,nypost,"Democrats melt down, call for Chuck Schumer’s ...",https://nypost.com/2025/11/10/us-news/democrat...,112
32,2025-11-10-22-49-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
84,132,2025-11-10-21-11-40 +0000,nypost,Trump pledges to ‘abide’ by deal to end govern...,https://nypost.com/2025/11/10/us-news/trump-pl...
215,68,2025-11-10-14-58-51 +0000,nypost,Supreme Court rejects Kim Davis bid to overtur...,https://nypost.com/2025/11/10/us-news/supreme-...
302,53,2025-11-10-05-22-38 +0000,wapo,Special counsel indicts former South Korean pr...,https://www.washingtonpost.com/world/2025/11/1...
189,53,2025-11-10-16-35-00 +0000,wsj,Sen. Bernie Sanders and a group of Democratic ...,https://www.wsj.com/politics/policy/the-politi...
201,52,2025-11-10-15-58-48 +0000,nypost,NYC realtor fired over controversial Zohran Ma...,https://nypost.com/2025/11/10/business/nyc-rea...
227,40,2025-11-10-14-01-26 +0000,nypost,Ex-French President Nicolas Sarkozy to be rele...,https://nypost.com/2025/11/10/world-news/franc...
119,40,2025-11-10-19-39-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...
152,39,2025-11-10-17-51-52 +0000,bbc,At least eight killed after car explodes near ...,https://www.bbc.com/news/articles/ceq0p8qnnrno...
270,38,2025-11-10-09-54-20 +0000,cbc,"Typhoon Fung-wong blows away from Philippines,...",https://www.cbc.ca/news/world/typhoon-fung-won...
254,32,2025-11-10-11-40-11 +0000,nypost,Trump threatens BBC with $1B lawsuit over doct...,https://nypost.com/2025/11/10/us-news/trump-la...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
